# Template für Topic Modeling
Dieses Template soll dabei helfen, Topic Modeling automatisiert und einheitlich durchzuführen. 

## Allgemeine Vorbereitungsschritte

In [ ]:
# Aktuelles Arbeitsverzeichnis anzeigen und bei Bedarf anpassen
# print(os.getcwd())
# os.chdir("C:/SV/HEX/Topic Modeling")

### Pakete laden

In [1]:
import pandas as pd
import stopwords
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
import sklearn
import re
import spacy
import numpy as np
import random
import torch
from bertopic.vectorizers import ClassTfidfTransformer
import os
import openpyxl
import optuna
from sklearn.cluster import KMeans
from bertopic.representation import MaximalMarginalRelevance

c:\Users\mhu\miniconda3\envs\bertopic-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Seed setzen
Wir setzen einen festen Seed, um Zufallszahlen in NumPy und PyTorch reproduzierbar zu machen, sowohl auf der CPU als auch auf der GPU (falls verfügbar). Das stellt sicher, dass Berechnungen mit zufälligen Operationen bei wiederholter Ausführung dieselben Ergebnisse liefern.

In [2]:
seed = 40  # Initialisiert den Seed-Wert für reproduzierbare Ergebnisse
np.random.seed(seed)  # Setzt den Seed für NumPy-Zufallszahlengeneratoren
random.seed(seed)  # Setzt den Seed für den Python-eigenen Zufallszahlengenerator
torch.manual_seed(seed)  # Setzt den Seed für PyTorch-Zufallszahlen
if torch.cuda.is_available():  # Überprüft, ob CUDA (GPU-Unterstützung) verfügbar ist
    torch.cuda.manual_seed_all(seed)  # Setzt den Seed für alle CUDA-Zufallszahlen (für GPU-Berechnungen)

### Datensätze einlesen
Der Trainings- und der Test-Datensatz werden hier eingelesen. Als Faustregel gilt, der Trainingsdatensatz sollte 80% und der Test-Datensatz 20% des Volumens ausmachen. 
Der Trainings-Datensatz wird für das trainieren / fitten des Modells verwendet. Der Test-Datensatz beinhaltet eine (in diesem Fall manuell erstellte) sogenannte "Ground Truth". Dies ist der Goldstandard, anhand dessen das Modell auf Performance hin überprüft wird. 

In [3]:
# Training-Datensatz
training_set = pd.read_csv("data/processed/train_data.csv")  # Liest die CSV-Datei ein und speichert sie in einem DataFrame
# training_set = training_set.sample(n=500, random_state=42)  # Zieht eine Zufallsstichprobe von 500 Zeilen aus dem DataFrame mit festgelegtem Seed für Reproduzierbarkeit
training_set = training_set.apply(lambda x: x.fillna('') if x.dtype == 'O' else x)  # Ersetzt fehlende Werte durch leere Strings in Objektspalten (Strings) und belässt numerische Spalten unverändert
training_set['titel_kursbesch'] = training_set['veranstaltung_titel'] + ' ' + training_set['kursbeschreibung']  # Kombiniert die Spalten "titel" und "kursbeschreibung" zu einer neuen Spalte "titel_kursbesch"
docs = training_set['titel_kursbesch'].tolist()  # Konvertiert die Inhalte der Spalte "titel_kursbesch" in eine Liste von Strings

In [5]:
# Test-Datensatz
test_data = pd.read_csv("C:/SV/HEX/Topic Modeling/test_set.csv", sep=";")
test_set = test_data["Volltext"]  # Texte der Test-Daten
ground_truth = test_data["Keywords"]  # Spalte im CSV mit manuell erstellten Begriffen, welche man als korrekt erachtet (i.d.R. einfach wichtige Wörter aus dem Text rauskopieren)

## NLP Vorbereitungsschritte
Zunächst werden die Trainingsdaten eingelesen und die gängigen Vorbereitungsschritte für NLP durchgeführt. Diese wären:
* Stopwords entfernen
* CountVectorizer spezifizieren

### Stopwords entfernen
Im Kontext des hier zu modellierenden Topic Modells werden sowohl standardisierte englische, deutsche als auch individuelle Stopwords generiert und im Objekt `sw` zusammengespielt.
Die Stopwords können je nach Anwendungsfall ergänzt oder reduziert werden.

In [4]:
irrelevant_terms = [
    "vl",
    "übung",
    "übungen",
    "seminar",
    "arbeitsgruppenseminar",
    "oberseminar",
    "proseminar",
    "blockveranstaltung",
    "vorlesung",
    "kolloquium",
    "theoriekolloquium",
    "einführung",
    "tutorium",
    "ue",
    "vereinbarung",
    "projekt",
    "praktikum",
    "masterprojekt",
    "wiederholerklausur",
    "fortgeschrittenenpraktikum",
    "hauptseminar",
    "fachpraktikum",
    "ergänzungsvorlesung",
    "forschungspraktikum",
    "begleitseminar",
    "abschlussarbeiten",
    "unterrichtspraktikum",
    "masterseminar",
    "proseminare",
    "praxisseminar",
    "praxissemester",
    "schulpraxis",
    "ringpraktikum",
    "basispraktikum",
    "praxistage",
    "industriepraktikum",
    "vorkurs",
    "projektseminar",
    "juniorprofessur",
    "masterarbeiten",
    "forschungsseminar",
    "modulbeschreibung",
    "veranstaltung",
    "kommentare",
    "raum",
    "uhrzeit",
    "vereinbarung",
    "vorlesung",



    # "Informatik","gleichnamigen", "übungsaufgaben", "vorlesungsstoff vertieft", "vorlesungsstoff", "grundlagen programmierung", "schriftliche übungsaufgaben", "vertieft schriftliche übungsaufgaben", "vorlesungsstoff vertieft schriftliche", "vertieft schriftliche", "lehre",
    # "siehe", "klausur", "inf misc", "sitzungen", "misc", "idee",


    # neue university stopwords 
"übung",
"beispiel",
"studierende",
"grundlage",
"laborübung",
"entwicklung",
"funktion",
"praktikum",
"versuch",
"methode",
"modell",
"kompetente",
"teil",
"ziel",
"schulungsreihe",
"thema",
"teilnehmer",
"bereich",
"präsentation",
"einführung",
"uhr",
"termin",
"überblick",
"aspekt",
"kontakt",
"anmeldung",
"inhalt",
"kontakt",
"sprechstunde",
"anforderung",
"modell",
"aufbau",
"fahren",
"student",
"jeweils",
"studierenden",
"prof",
"seminar",
"seminare",
"–",
"fortlaufendes",
"schwerpunktmodul",
"entwickelt",
"handelns",
"ansatz",
"schwerpunktmoduls",
"einzubringen",
"semestern",
"semesternseminar",
"seminarnseminar",
"zulassung",
"rahmen",
"blick",
"bereit",
"teilnehmenden",
"gruppe",
"teilnehmerinnen",
"modul",
"ausdruck",
"einfluß",
"kolloquium",
"thematisiert",
"bereiche",
"vermittelt",
"anhand",
"schwerpunkt",
"seminars",
"lehrperson",
"deutsche",
"form",
"gruppen",
"gegenstand",
"anliegen",
"ansätzen",
"vorgestellt",
"ndie",
"forschungscolleges",
"erprobt",
"denkens",
"ansatzes",
"vorausgesetzt",
"montag",
"studium",
"blockseminar",
"veranstaltung",
"masterstudierende",
"datum",
"immatrikulierte",
"dienstagskolloquium",
"leistungsnachweise",
"leistungsnachweis",
"vgl",
"the",
"studiengangs",
"veranstaltung",
"seminar",
"vorlesung",
"kurs",
"modul",
"angebot",
"behandlung",
"themen",
"einführung",
"grundlagen",
"aufbau",
"überblick",
"inhalt",
"praxis",
"theorie",
"schwerpunkt",
"recht",
"rechts",
"rechtsgebiete",
"rechtswissenschaft",
"rechtsfragen",
"bereich",
"gebiete",
"gesetz",
"juristisch",
"aktuell",
"neue",
"verschiedenen",
"interdisziplinär",
"relevant",
"bedeutung",
"besonders",
"werden",
"sollen",
"kann",
"bietet",
"z.b.",
"u.a.",
"etc.",
"bspw.",
"ggf.",
"i.d.r.",
"family",
"relationship",
"practice",
"lesson",
"better",
"lawsuits",
"they",
"aid",
"united",
"register",
"between",
"language",
"about",
"einfache",
"right",
"acp",
"before",
"common",
"please",
"matrikel",
"registration",
"jurisdictions",
"across",
"limits",
"bundesverfassungsgerichts",
"künftigen",
"naturrecht",
"recent",
"schriftliches",
"literaturverarbeitung",
"kurzvortrag",
"genügt",
"seelmann",
"referats",
"lage",
"fighting",
"neuster",
"arbeitsmethoden",
"eugh",
"algorithmus",
"formel",
"adresse",
"resolution",
"dispute",
"module",
"literaturhinweise",
"civil",
"regulatory",
"menschenwürde",
"machthabern",


    # solariz stopwords 
    "ab",
  "aber",
  "abermaliges",
  "abermals",
  "abgerufen",
  "abgerufene",
  "abgerufener",
  "abgerufenes",
  "abgesehen",
  "acht",
  "aehnlich",
  "aehnliche",
  "aehnlichem",
  "aehnlichen",
  "aehnlicher",
  "aehnliches",
  "aehnlichste",
  "aehnlichstem",
  "aehnlichsten",
  "aehnlichster",
  "aehnlichstes",
  "aeusserst",
  "aeusserste",
  "aeusserstem",
  "aeussersten",
  "aeusserster",
  "aeusserstes",
  "ähnlich",
  "ähnliche",
  "ähnlichem",
  "ähnlichen",
  "ähnlicher",
  "ähnliches",
  "ähnlichst",
  "ähnlichste",
  "ähnlichstem",
  "ähnlichsten",
  "ähnlichster",
  "ähnlichstes",
  "alle",
  "allein",
  "alleine",
  "allem",
  "allemal",
  "allen",
  "allenfalls",
  "allenthalben",
  "aller",
  "allerdings",
  "allerlei",
  "alles",
  "allesamt",
  "allg",
  "allg.",
  "allgemein",
  "allgemeine",
  "allgemeinem",
  "allgemeinen",
  "allgemeiner",
  "allgemeines",
  "allgemeinste",
  "allgemeinstem",
  "allgemeinsten",
  "allgemeinster",
  "allgemeinstes",
  "allmählich",
  "allzeit",
  "allzu",
  "als",
  "alsbald",
  "also",
  "am",
  "an",
  "and",
  "andauernd",
  "andauernde",
  "andauerndem",
  "andauernden",
  "andauernder",
  "andauerndes",
  "ander",
  "andere",
  "anderem",
  "anderen",
  "anderenfalls",
  "anderer",
  "andererseits",
  "anderes",
  "anderm",
  "andern",
  "andernfalls",
  "anderr",
  "anders",
  "anderst",
  "anderweitig",
  "anderweitige",
  "anderweitigem",
  "anderweitigen",
  "anderweitiger",
  "anderweitiges",
  "anerkannt",
  "anerkannte",
  "anerkannter",
  "anerkanntes",
  "anfangen",
  "anfing",
  "angefangen",
  "angesetze",
  "angesetzt",
  "angesetzten",
  "angesetzter",
  "ans",
  "anscheinend",
  "ansetzen",
  "ansonst",
  "ansonsten",
  "anstatt",
  "anstelle",
  "arbeiten",
  "auch",
  "auf",
  "aufgehört",
  "aufgrund",
  "aufhören",
  "aufhörte",
  "aufzusuchen",
  "augenscheinlich",
  "augenscheinliche",
  "augenscheinlichem",
  "augenscheinlichen",
  "augenscheinlicher",
  "augenscheinliches",
  "augenscheinlichst",
  "augenscheinlichste",
  "augenscheinlichstem",
  "augenscheinlichsten",
  "augenscheinlichster",
  "augenscheinlichstes",
  "aus",
  "ausdrücken",
  "ausdrücklich",
  "ausdrückliche",
  "ausdrücklichem",
  "ausdrücklichen",
  "ausdrücklicher",
  "ausdrückliches",
  "ausdrückt",
  "ausdrückte",
  "ausgenommen",
  "ausgenommene",
  "ausgenommenem",
  "ausgenommenen",
  "ausgenommener",
  "ausgenommenes",
  "ausgerechnet",
  "ausgerechnete",
  "ausgerechnetem",
  "ausgerechneten",
  "ausgerechneter",
  "ausgerechnetes",
  "ausnahmslos",
  "ausnahmslose",
  "ausnahmslosem",
  "ausnahmslosen",
  "ausnahmsloser",
  "ausnahmsloses",
  "außen",
  "ausser",
  "ausserdem",
  "außerhalb",
  "äusserst",
  "äusserste",
  "äusserstem",
  "äussersten",
  "äusserster",
  "äusserstes",
  "author",
  "autor",
  "baelde",
  "bald",
  "bälde",
  "bearbeite",
  "bearbeiten",
  "bearbeitete",
  "bearbeiteten",
  "bedarf",
  "bedürfen",
  "bedurfte",
  "been",
  "befahl",
  "befiehlt",
  "befiehlte",
  "befohlene",
  "befohlens",
  "befragen",
  "befragte",
  "befragten",
  "befragter",
  "begann",
  "beginnen",
  "begonnen",
  "behalten",
  "behielt",
  "bei",
  "beide",
  "beidem",
  "beiden",
  "beider",
  "beiderlei",
  "beides",
  "beim",
  "beinahe",
  "beisammen",
  "beispielsweise",
  "beitragen",
  "beitrugen",
  "bekannt",
  "bekannte",
  "bekannter",
  "bekanntlich",
  "bekanntliche",
  "bekanntlichem",
  "bekanntlichen",
  "bekanntlicher",
  "bekanntliches",
  "bekennen",
  "benutzt",
  "bereits",
  "berichten",
  "berichtet",
  "berichtete",
  "berichteten",
  "besonders",
  "besser",
  "bessere",
  "besserem",
  "besseren",
  "besserer",
  "besseres",
  "bestehen",
  "besteht",
  "bestenfalls",
  "bestimmt",
  "bestimmte",
  "bestimmtem",
  "bestimmten",
  "bestimmter",
  "bestimmtes",
  "beträchtlich",
  "beträchtliche",
  "beträchtlichem",
  "beträchtlichen",
  "beträchtlicher",
  "beträchtliches",
  "betraechtlich",
  "betraechtliche",
  "betraechtlichem",
  "betraechtlichen",
  "betraechtlicher",
  "betraechtliches",
  "betreffend",
  "betreffende",
  "betreffendem",
  "betreffenden",
  "betreffender",
  "betreffendes",
  "bevor",
  "bez",
  "bez.",
  "bezgl",
  "bezgl.",
  "bezueglich",
  "bezüglich",
  "bietet",
  "bin",
  "bis",
  "bisher",
  "bisherige",
  "bisherigem",
  "bisherigen",
  "bisheriger",
  "bisheriges",
  "bislang",
  "bisschen",
  "bist",
  "bitte",
  "bleiben",
  "bleibt",
  "blieb",
  "bloss",
  "böden",
  "boeden",
  "brachte",
  "brachten",
  "brauchen",
  "braucht",
  "bräuchte",
  "bringen",
  "bsp",
  "bsp.",
  "bspw",
  "bspw.",
  "bzw",
  "bzw.",
  "ca",
  "ca.",
  "circa",
  "da",
  "dabei",
  "dadurch",
  "dafuer",
  "dafür",
  "dagegen",
  "daher",
  "dahin",
  "dahingehend",
  "dahingehende",
  "dahingehendem",
  "dahingehenden",
  "dahingehender",
  "dahingehendes",
  "dahinter",
  "damalige",
  "damaligem",
  "damaligen",
  "damaliger",
  "damaliges",
  "damals",
  "damit",
  "danach",
  "daneben",
  "dank",
  "danke",
  "danken",
  "dann",
  "dannen",
  "daran",
  "darauf",
  "daraus",
  "darf",
  "darfst",
  "darin",
  "darüber",
  "darüberhinaus",
  "darueber",
  "darueberhinaus",
  "darum",
  "darunter",
  "das",
  "daß",
  "dass",
  "dasselbe",
  "Dat",
  "davon",
  "davor",
  "dazu",
  "dazwischen",
  "dein",
  "deine",
  "deinem",
  "deinen",
  "deiner",
  "deines",
  "dem",
  "demgegenüber",
  "demgegenueber",
  "demgemaess",
  "demgemäss",
  "demnach",
  "demselben",
  "den",
  "denen",
  "denkbar",
  "denkbare",
  "denkbarem",
  "denkbaren",
  "denkbarer",
  "denkbares",
  "denn",
  "dennoch",
  "denselben",
  "der",
  "derart",
  "derartig",
  "derartige",
  "derartigem",
  "derartigen",
  "derartiger",
  "derem",
  "deren",
  "derer",
  "derjenige",
  "derjenigen",
  "derselbe",
  "derselben",
  "derzeit",
  "derzeitig",
  "derzeitige",
  "derzeitigem",
  "derzeitigen",
  "derzeitiges",
  "des",
  "deshalb",
  "desselben",
  "dessen",
  "dessenungeachtet",
  "desto",
  "desungeachtet",
  "deswegen",
  "dich",
  "die",
  "diejenige",
  "diejenigen",
  "dies",
  "diese",
  "dieselbe",
  "dieselben",
  "diesem",
  "diesen",
  "dieser",
  "dieses",
  "diesseitig",
  "diesseitige",
  "diesseitigem",
  "diesseitigen",
  "diesseitiger",
  "diesseitiges",
  "diesseits",
  "dinge",
  "dir",
  "direkt",
  "direkte",
  "direkten",
  "direkter",
  "doch",
  "doppelt",
  "dort",
  "dorther",
  "dorthin",
  "dran",
  "drauf",
  "drei",
  "dreißig",
  "drin",
  "dritte",
  "drüber",
  "drueber",
  "drum",
  "drunter",
  "du",
  "duerfte",
  "duerften",
  "duerftest",
  "duerftet",
  "dunklen",
  "durch",
  "durchaus",
  "durchweg",
  "durchwegs",
  "dürfen",
  "durfte",
  "dürfte",
  "durften",
  "dürften",
  "durftest",
  "dürftest",
  "durftet",
  "dürftet",
  "eben",
  "ebenfalls",
  "ebenso",
  "ect",
  "ect.",
  "ehe",
  "eher",
  "eheste",
  "ehestem",
  "ehesten",
  "ehester",
  "ehestes",
  "eigen",
  "eigene",
  "eigenem",
  "eigenen",
  "eigener",
  "eigenes",
  "eigenst",
  "eigentlich",
  "eigentliche",
  "eigentlichem",
  "eigentlichen",
  "eigentlicher",
  "eigentliches",
  "ein",
  "einbaün",
  "eine",
  "einem",
  "einen",
  "einer",
  "einerlei",
  "einerseits",
  "eines",
  "einfach",
  "einführen",
  "einführte",
  "einführten",
  "eingesetzt",
  "einig",
  "einige",
  "einigem",
  "einigen",
  "einiger",
  "einigermaßen",
  "einiges",
  "einmal",
  "einmalig",
  "einmalige",
  "einmaligem",
  "einmaligen",
  "einmaliger",
  "einmaliges",
  "eins",
  "einseitig",
  "einseitige",
  "einseitigen",
  "einseitiger",
  "einst",
  "einstmals",
  "einzig",
  "empfunden",
  "ende",
  "entgegen",
  "entlang",
  "entsprechend",
  "entsprechende",
  "entsprechendem",
  "entsprechenden",
  "entsprechender",
  "entsprechendes",
  "entweder",
  "er",
  "ergänze",
  "ergänzen",
  "ergänzte",
  "ergänzten",
  "ergo",
  "erhält",
  "erhalten",
  "erhielt",
  "erhielten",
  "erneut",
  "eröffne",
  "eröffnen",
  "eröffnet",
  "eröffnete",
  "eröffnetes",
  "erscheinen",
  "erst",
  "erste",
  "erstem",
  "ersten",
  "erster",
  "erstere",
  "ersterem",
  "ersteren",
  "ersterer",
  "ersteres",
  "erstes",
  "es",
  "etc",
  "etc.",
  "etliche",
  "etlichem",
  "etlichen",
  "etlicher",
  "etliches",
  "etwa",
  "etwaige",
  "etwas",
  "euch",
  "euer",
  "eure",
  "eurem",
  "euren",
  "eurer",
  "eures",
  "euretwegen",
  "fall",
  "falls",
  "fand",
  "fast",
  "ferner",
  "fertig",
  "finde",
  "finden",
  "findest",
  "findet",
  "folgend",
  "folgende",
  "folgendem",
  "folgenden",
  "folgender",
  "folgendermassen",
  "folgendes",
  "folglich",
  "for",
  "fordern",
  "fordert",
  "forderte",
  "forderten",
  "fort",
  "fortsetzen",
  "fortsetzt",
  "fortsetzte",
  "fortsetzten",
  "fragte",
  "frau",
  "frei",
  "freie",
  "freier",
  "freies",
  "fuer",
  "fuers",
  "fünf",
  "für",
  "fürs",
  "gab",
  "gaenzlich",
  "gaenzliche",
  "gaenzlichem",
  "gaenzlichen",
  "gaenzlicher",
  "gaenzliches",
  "gängig",
  "gängige",
  "gängigen",
  "gängiger",
  "gängiges",
  "ganz",
  "ganze",
  "ganzem",
  "ganzen",
  "ganzer",
  "ganzes",
  "gänzlich",
  "gänzliche",
  "gänzlichem",
  "gänzlichen",
  "gänzlicher",
  "gänzliches",
  "gar",
  "gbr",
  "geb",
  "geben",
  "geblieben",
  "gebracht",
  "gedurft",
  "geehrt",
  "geehrte",
  "geehrten",
  "geehrter",
  "gefallen",
  "gefälligst",
  "gefällt",
  "gefiel",
  "gegeben",
  "gegen",
  "gegenüber",
  "gegenueber",
  "gehabt",
  "gehalten",
  "gehen",
  "geht",
  "gekommen",
  "gekonnt",
  "gemacht",
  "gemaess",
  "gemäss",
  "gemeinhin",
  "gemocht",
  "genau",
  "genommen",
  "genug",
  "gepriesener",
  "gepriesenes",
  "gerade",
  "gern",
  "gesagt",
  "gesehen",
  "gestern",
  "gestrige",
  "getan",
  "geteilt",
  "geteilte",
  "getragen",
  "getrennt",
  "gewesen",
  "gewiss",
  "gewisse",
  "gewissem",
  "gewissen",
  "gewisser",
  "gewissermaßen",
  "gewisses",
  "gewollt",
  "geworden",
  "ggf",
  "ggf.",
  "gib",
  "gibt",
  "gilt",
  "gleich",
  "gleiche",
  "gleichem",
  "gleichen",
  "gleicher",
  "gleiches",
  "gleichsam",
  "gleichste",
  "gleichstem",
  "gleichsten",
  "gleichster",
  "gleichstes",
  "gleichwohl",
  "gleichzeitig",
  "gleichzeitige",
  "gleichzeitigem",
  "gleichzeitigen",
  "gleichzeitiger",
  "gleichzeitiges",
  "glücklicherweise",
  "gluecklicherweise",
  "gmbh",
  "gottseidank",
  "gratulieren",
  "gratuliert",
  "gratulierte",
  "groesstenteils",
  "grösstenteils",
  "gruendlich",
  "gründlich",
  "gut",
  "gute",
  "guten",
  "hab",
  "habe",
  "haben",
  "habt",
  "haette",
  "haeufig",
  "haeufige",
  "haeufigem",
  "haeufigen",
  "haeufiger",
  "haeufigere",
  "haeufigeren",
  "haeufigerer",
  "haeufigeres",
  "halb",
  "hallo",
  "halten",
  "hast",
  "hat",
  "hätt",
  "hatte",
  "hätte",
  "hatten",
  "hätten",
  "hattest",
  "hattet",
  "häufig",
  "häufige",
  "häufigem",
  "häufigen",
  "häufiger",
  "häufigere",
  "häufigeren",
  "häufigerer",
  "häufigeres",
  "hen",
  "her",
  "heraus",
  "herein",
  "herum",
  "heute",
  "heutige",
  "heutigem",
  "heutigen",
  "heutiger",
  "heutiges",
  "hier",
  "hierbei",
  "hiermit",
  "hiesige",
  "hiesigem",
  "hiesigen",
  "hiesiger",
  "hiesiges",
  "hin",
  "hindurch",
  "hinein",
  "hingegen",
  "hinlanglich",
  "hinlänglich",
  "hinten",
  "hintendran",
  "hinter",
  "hinterher",
  "hinterm",
  "hintern",
  "hinunter",
  "hoch",
  "höchst",
  "höchstens",
  "http",
  "hundert",
  "ich",
  "igitt",
  "ihm",
  "ihn",
  "ihnen",
  "ihr",
  "ihre",
  "ihrem",
  "ihren",
  "ihrer",
  "ihres",
  "ihretwegen",
  "ihrige",
  "ihrigen",
  "ihriges",
  "im",
  "immer",
  "immerhin",
  "immerwaehrend",
  "immerwaehrende",
  "immerwaehrendem",
  "immerwaehrenden",
  "immerwaehrender",
  "immerwaehrendes",
  "immerwährend",
  "immerwährende",
  "immerwährendem",
  "immerwährenden",
  "immerwährender",
  "immerwährendes",
  "immerzu",
  "important",
  "in",
  "indem",
  "indessen",
  "Inf.",
  "info",
  "infolge",
  "infolgedessen",
  "information",
  "innen",
  "innerhalb",
  "innerlich",
  "ins",
  "insbesondere",
  "insgeheim",
  "insgeheime",
  "insgeheimer",
  "insgesamt",
  "insgesamte",
  "insgesamter",
  "insofern",
  "inzwischen",
  "irgend",
  "irgendein",
  "irgendeine",
  "irgendeinem",
  "irgendeiner",
  "irgendeines",
  "irgendetwas",
  "irgendjemand",
  "irgendjemandem",
  "irgendwann",
  "irgendwas",
  "irgendwelche",
  "irgendwen",
  "irgendwenn",
  "irgendwer",
  "irgendwie",
  "irgendwo",
  "irgendwohin",
  "ist",
  "ja",
  "jaehrig",
  "jaehrige",
  "jaehrigem",
  "jaehrigen",
  "jaehriger",
  "jaehriges",
  "jährig",
  "jährige",
  "jährigem",
  "jährigen",
  "jähriges",
  "je",
  "jede",
  "jedem",
  "jeden",
  "jedenfalls",
  "jeder",
  "jederlei",
  "jedes",
  "jedesmal",
  "jedoch",
  "jeglichem",
  "jeglichen",
  "jeglicher",
  "jegliches",
  "jemals",
  "jemand",
  "jemandem",
  "jemanden",
  "jemandes",
  "jene",
  "jenem",
  "jenen",
  "jener",
  "jenes",
  "jenseitig",
  "jenseitigem",
  "jenseitiger",
  "jenseits",
  "jetzt",
  "jung",
  "junge",
  "jungem",
  "jungen",
  "junger",
  "junges",
  "kaeumlich",
  "kam",
  "kann",
  "kannst",
  "kaum",
  "käumlich",
  "kein",
  "keine",
  "keinem",
  "keinen",
  "keiner",
  "keinerlei",
  "keines",
  "keineswegs",
  "klar",
  "klare",
  "klaren",
  "klares",
  "klein",
  "kleinen",
  "kleiner",
  "kleines",
  "koennen",
  "koennt",
  "koennte",
  "koennten",
  "koenntest",
  "koenntet",
  "komme",
  "kommen",
  "kommt",
  "konkret",
  "konkrete",
  "konkreten",
  "konkreter",
  "konkretes",
  "könn",
  "können",
  "könnt",
  "konnte",
  "könnte",
  "konnten",
  "könnten",
  "konntest",
  "könntest",
  "konntet",
  "könntet",
  "kuenftig",
  "kuerzlich",
  "kuerzlichst",
  "künftig",
  "kürzlich",
  "kürzlichst",
  "laengst",
  "lag",
  "lagen",
  "langsam",
  "längst",
  "längstens",
  "lassen",
  "laut",
  "lediglich",
  "leer",
  "legen",
  "legte",
  "legten",
  "leicht",
  "leider",
  "lesen",
  "letze",
  "letzte",
  "letzten",
  "letztendlich",
  "letztens",
  "letztere",
  "letzterem",
  "letzterer",
  "letzteres",
  "letztes",
  "letztlich",
  "lichten",
  "liegt",
  "liest",
  "links",
  "mache",
  "machen",
  "machst",
  "macht",
  "machte",
  "machten",
  "mag",
  "magst",
  "mal",
  "man",
  "manch",
  "manche",
  "manchem",
  "manchen",
  "mancher",
  "mancherlei",
  "mancherorts",
  "manches",
  "manchmal",
  "mann",
  "margin",
  "massgebend",
  "massgebende",
  "massgebendem",
  "massgebenden",
  "massgebender",
  "massgebendes",
  "massgeblich",
  "massgebliche",
  "massgeblichem",
  "massgeblichen",
  "massgeblicher",
  "mehr",
  "mehrere",
  "mehrerer",
  "mehrfach",
  "mehrmalig",
  "mehrmaligem",
  "mehrmaliger",
  "mehrmaliges",
  "mein",
  "meine",
  "meinem",
  "meinen",
  "meiner",
  "meines",
  "meinetwegen",
  "meins",
  "meist",
  "meiste",
  "meisten",
  "meistens",
  "meistenteils",
  "meta",
  "mich",
  "mindestens",
  "mir",
  "mit",
  "miteinander",
  "mitgleich",
  "mithin",
  "mitnichten",
  "mittels",
  "mittelst",
  "mitten",
  "mittig",
  "mitunter",
  "mitwohl",
  "mochte",
  "möchte",
  "möchten",
  "möchtest",
  "moechte",
  "moeglich",
  "moeglichst",
  "moeglichste",
  "moeglichstem",
  "moeglichsten",
  "moeglichster",
  "mögen",
  "möglich",
  "mögliche",
  "möglichen",
  "möglicher",
  "möglicherweise",
  "möglichst",
  "möglichste",
  "möglichstem",
  "möglichsten",
  "möglichster",
  "morgen",
  "morgige",
  "muessen",
  "muesst",
  "muesste",
  "muss",
  "müssen",
  "musst",
  "müßt",
  "musste",
  "müsste",
  "mussten",
  "müssten",
  "nach",
  "nachdem",
  "nacher",
  "nachher",
  "nachhinein",
  "nächste",
  "nacht",
  "naechste",
  "naemlich",
  "nahm",
  "nämlich",
  "naturgemaess",
  "naturgemäss",
  "natürlich",
  "ncht",
  "neben",
  "nebenan",
  "nehmen",
  "nein",
  "neu",
  "neue",
  "neuem",
  "neuen",
  "neuer",
  "neuerdings",
  "neuerlich",
  "neuerliche",
  "neuerlichem",
  "neuerlicher",
  "neuerliches",
  "neues",
  "neulich",
  "neun",
  "nicht",
  "nichts",
  "nichtsdestotrotz",
  "nichtsdestoweniger",
  "nie",
  "niemals",
  "niemand",
  "niemandem",
  "niemanden",
  "niemandes",
  "nimm",
  "nimmer",
  "nimmt",
  "nirgends",
  "nirgendwo",
  "noch",
  "noetigenfalls",
  "nötigenfalls",
  "nun",
  "nur",
  "nutzen",
  "nutzt",
  "nützt",
  "nutzung",
  "ob",
  "oben",
  "ober",
  "oberen",
  "oberer",
  "oberhalb",
  "oberste",
  "obersten",
  "oberster",
  "obgleich",
  "obs",
  "obschon",
  "obwohl",
  "oder",
  "oefter",
  "oefters",
  "off",
  "offenkundig",
  "offenkundige",
  "offenkundigem",
  "offenkundigen",
  "offenkundiger",
  "offenkundiges",
  "offensichtlich",
  "offensichtliche",
  "offensichtlichem",
  "offensichtlichen",
  "offensichtlicher",
  "offensichtliches",
  "oft",
  "öfter",
  "öfters",
  "oftmals",
  "ohne",
  "ohnedies",
  "online",
  "paar",
  "partout",
  "per",
  "persoenlich",
  "persoenliche",
  "persoenlichem",
  "persoenlicher",
  "persoenliches",
  "persönlich",
  "persönliche",
  "persönlicher",
  "persönliches",
  "pfui",
  "ploetzlich",
  "ploetzliche",
  "ploetzlichem",
  "ploetzlicher",
  "ploetzliches",
  "plötzlich",
  "plötzliche",
  "plötzlichem",
  "plötzlicher",
  "plötzliches",
  "pro",
  "quasi",
  "reagiere",
  "reagieren",
  "reagiert",
  "reagierte",
  "recht",
  "rechts",
  "regelmäßig",
  "reichlich",
  "reichliche",
  "reichlichem",
  "reichlichen",
  "reichlicher",
  "restlos",
  "restlose",
  "restlosem",
  "restlosen",
  "restloser",
  "restloses",
  "richtiggehend",
  "richtiggehende",
  "richtiggehendem",
  "richtiggehenden",
  "richtiggehender",
  "richtiggehendes",
  "rief",
  "rund",
  "rundheraus",
  "rundum",
  "runter",
  "sage",
  "sagen",
  "sagt",
  "sagte",
  "sagten",
  "sagtest",
  "sagtet",
  "samt",
  "sämtliche",
  "sang",
  "sangen",
  "sattsam",
  "schätzen",
  "schätzt",
  "schätzte",
  "schätzten",
  "scheinbar",
  "scheinen",
  "schlechter",
  "schlicht",
  "schlichtweg",
  "schließlich",
  "schlussendlich",
  "schnell",
  "schon",
  "schreibe",
  "schreiben",
  "schreibens",
  "schreiber",
  "schwerlich",
  "schwerliche",
  "schwerlichem",
  "schwerlichen",
  "schwerlicher",
  "schwerliches",
  "schwierig",
  "sechs",
  "sect",
  "sehe",
  "sehen",
  "sehr",
  "sehrwohl",
  "seht",
  "sei",
  "seid",
  "seien",
  "seiest",
  "seiet",
  "sein",
  "seine",
  "seinem",
  "seinen",
  "seiner",
  "seines",
  "seit",
  "seitdem",
  "seite",
  "seiten",
  "seither",
  "selbe",
  "selben",
  "selber",
  "selbst",
  "selbstredend",
  "selbstredende",
  "selbstredendem",
  "selbstredenden",
  "selbstredender",
  "selbstredendes",
  "seltsamerweise",
  "senke",
  "senken",
  "senkt",
  "senkte",
  "senkten",
  "setzen",
  "setzt",
  "setzte",
  "setzten",
  "sich",
  "sicher",
  "sicherlich",
  "sie",
  "sieben",
  "siebte",
  "siehe",
  "sieht",
  "sind",
  "singen",
  "singt",
  "so",
  "sobald",
  "sodaß",
  "soeben",
  "sofern",
  "sofort",
  "sog",
  "sogar",
  "sogleich",
  "solange",
  "solc",
  "solc hen",
  "solch",
  "solche",
  "solchem",
  "solchen",
  "solcher",
  "solches",
  "soll",
  "sollen",
  "sollst",
  "sollt",
  "sollte",
  "sollten",
  "solltest",
  "solltet",
  "somit",
  "sondern",
  "sonst",
  "sonstig",
  "sonstige",
  "sonstigem",
  "sonstiger",
  "sonstwo",
  "sooft",
  "soviel",
  "soweit",
  "sowie",
  "sowieso",
  "sowohl",
  "später",
  "spielen",
  "startet",
  "startete",
  "starteten",
  "statt",
  "stattdessen",
  "steht",
  "steige",
  "steigen",
  "steigt",
  "stellenweise",
  "stellenweisem",
  "stellenweisen",
  "stets",
  "stieg",
  "stiegen",
  "such",
  "suchen",
  "tages",
  "tat",
  "tät",
  "tatsächlich",
  "tatsächlichen",
  "tatsächlicher",
  "tatsächliches",
  "tatsaechlich",
  "tatsaechlichen",
  "tatsaechlicher",
  "tatsaechliches",
  "tausend",
  "teile",
  "teilen",
  "teilte",
  "teilten",
  "tief",
  "titel",
  "toll",
  "total",
  "trage",
  "tragen",
  "trägt",
  "trotzdem",
  "trug",
  "tun",
  "tust",
  "tut",
  "txt",
  "übel",
  "über",
  "überall",
  "überallhin",
  "überaus",
  "überdies",
  "überhaupt",
  "überll",
  "übermorgen",
  "üblicherweise",
  "übrig",
  "übrigens",
  "ueber",
  "ueberall",
  "ueberallhin",
  "ueberaus",
  "ueberdies",
  "ueberhaupt",
  "uebermorgen",
  "ueblicherweise",
  "uebrig",
  "uebrigens",
  "um",
  "ums",
  "umso",
  "umstaendehalber",
  "umständehalber",
  "unbedingt",
  "unbedingte",
  "unbedingter",
  "unbedingtes",
  "und",
  "unerhoert",
  "unerhoerte",
  "unerhoertem",
  "unerhoerten",
  "unerhoerter",
  "unerhoertes",
  "unerhört",
  "unerhörte",
  "unerhörtem",
  "unerhörten",
  "unerhörter",
  "unerhörtes",
  "ungefähr",
  "ungemein",
  "ungewoehnlich",
  "ungewoehnliche",
  "ungewoehnlichem",
  "ungewoehnlichen",
  "ungewoehnlicher",
  "ungewoehnliches",
  "ungewöhnlich",
  "ungewöhnliche",
  "ungewöhnlichem",
  "ungewöhnlichen",
  "ungewöhnlicher",
  "ungewöhnliches",
  "ungleich",
  "ungleiche",
  "ungleichem",
  "ungleichen",
  "ungleicher",
  "ungleiches",
  "unmassgeblich",
  "unmassgebliche",
  "unmassgeblichem",
  "unmassgeblichen",
  "unmassgeblicher",
  "unmassgebliches",
  "unmoeglich",
  "unmoegliche",
  "unmoeglichem",
  "unmoeglichen",
  "unmoeglicher",
  "unmoegliches",
  "unmöglich",
  "unmögliche",
  "unmöglichen",
  "unmöglicher",
  "unnötig",
  "uns",
  "unsaeglich",
  "unsaegliche",
  "unsaeglichem",
  "unsaeglichen",
  "unsaeglicher",
  "unsaegliches",
  "unsagbar",
  "unsagbare",
  "unsagbarem",
  "unsagbaren",
  "unsagbarer",
  "unsagbares",
  "unsäglich",
  "unsägliche",
  "unsäglichem",
  "unsäglichen",
  "unsäglicher",
  "unsägliches",
  "unse",
  "unsem",
  "unsen",
  "unser",
  "unsere",
  "unserem",
  "unseren",
  "unserer",
  "unseres",
  "unserm",
  "unses",
  "unsre",
  "unsrem",
  "unsren",
  "unsrer",
  "unsres",
  "unstreitig",
  "unstreitige",
  "unstreitigem",
  "unstreitigen",
  "unstreitiger",
  "unstreitiges",
  "unten",
  "unter",
  "unterbrach",
  "unterbrechen",
  "untere",
  "unterem",
  "unteres",
  "unterhalb",
  "unterste",
  "unterster",
  "unterstes",
  "unwichtig",
  "unzweifelhaft",
  "unzweifelhafte",
  "unzweifelhaftem",
  "unzweifelhaften",
  "unzweifelhafter",
  "unzweifelhaftes",
  "usw",
  "usw.",
  "vergangen",
  "vergangene",
  "vergangener",
  "vergangenes",
  "vermag",
  "vermögen",
  "vermutlich",
  "vermutliche",
  "vermutlichem",
  "vermutlichen",
  "vermutlicher",
  "vermutliches",
  "veröffentlichen",
  "veröffentlicher",
  "veröffentlicht",
  "veröffentlichte",
  "veröffentlichten",
  "veröffentlichtes",
  "verrate",
  "verraten",
  "verriet",
  "verrieten",
  "version",
  "versorge",
  "versorgen",
  "versorgt",
  "versorgte",
  "versorgten",
  "versorgtes",
  "viel",
  "viele",
  "vielen",
  "vieler",
  "vielerlei",
  "vieles",
  "vielleicht",
  "vielmalig",
  "vielmals",
  "vier",
  "voellig",
  "voellige",
  "voelligem",
  "voelligen",
  "voelliger",
  "voelliges",
  "voelligst",
  "vollends",
  "völlig",
  "völlige",
  "völligem",
  "völligen",
  "völliger",
  "völliges",
  "völligst",
  "vollstaendig",
  "vollstaendige",
  "vollstaendigem",
  "vollstaendigen",
  "vollstaendiger",
  "vollstaendiges",
  "vollständig",
  "vollständige",
  "vollständigem",
  "vollständigen",
  "vollständiger",
  "vollständiges",
  "vom",
  "von",
  "vor",
  "voran",
  "vorbei",
  "vorgestern",
  "vorher",
  "vorherig",
  "vorherige",
  "vorherigem",
  "vorheriger",
  "vorne",
  "vorüber",
  "vorueber",
  "wachen",
  "waehrend",
  "waehrenddessen",
  "waere",
  "während",
  "währenddessen",
  "wann",
  "war",
  "wär",
  "wäre",
  "waren",
  "wären",
  "warst",
  "wart",
  "warum",
  "was",
  "weder",
  "weg",
  "wegen",
  "weil",
  "weiß",
  "weit",
  "weiter",
  "weitere",
  "weiterem",
  "weiteren",
  "weiterer",
  "weiteres",
  "weiterhin",
  "weitestgehend",
  "weitestgehende",
  "weitestgehendem",
  "weitestgehenden",
  "weitestgehender",
  "weitestgehendes",
  "weitgehend",
  "weitgehende",
  "weitgehendem",
  "weitgehenden",
  "weitgehender",
  "weitgehendes",
  "welche",
  "welchem",
  "welchen",
  "welcher",
  "welches",
  "wem",
  "wen",
  "wenig",
  "wenige",
  "weniger",
  "wenigstens",
  "wenn",
  "wenngleich",
  "wer",
  "werde",
  "werden",
  "werdet",
  "weshalb",
  "wessen",
  "weswegen",
  "wichtig",
  "wie",
  "wieder",
  "wiederum",
  "wieso",
  "wieviel",
  "wieviele",
  "wievieler",
  "wiewohl",
  "will",
  "willst",
  "wir",
  "wird",
  "wirklich",
  "wirklichem",
  "wirklicher",
  "wirkliches",
  "wirst",
  "wo",
  "wobei",
  "wodurch",
  "wofuer",
  "wofür",
  "wogegen",
  "woher",
  "wohin",
  "wohingegen",
  "wohl",
  "wohlgemerkt",
  "wohlweislich",
  "wolle",
  "wollen",
  "wollt",
  "wollte",
  "wollten",
  "wolltest",
  "wolltet",
  "womit",
  "womoeglich",
  "womoegliche",
  "womoeglichem",
  "womoeglichen",
  "womoeglicher",
  "womoegliches",
  "womöglich",
  "womögliche",
  "womöglichem",
  "womöglichen",
  "womöglicher",
  "womögliches",
  "woran",
  "woraufhin",
  "woraus",
  "worden",
  "worin",
  "wuerde",
  "wuerden",
  "wuerdest",
  "wuerdet",
  "wurde",
  "würde",
  "wurden",
  "würden",
  "wurdest",
  "würdest",
  "wurdet",
  "würdet",
  "www",
  "x",
  "z.B.",
  "zahlreich",
  "zahlreichem",
  "zahlreicher",
  "zB",
  "zb.",
  "zehn",
  "zeitweise",
  "zeitweisem",
  "zeitweisen",
  "zeitweiser",
  "ziehen",
  "zieht",
  "ziemlich",
  "ziemliche",
  "ziemlichem",
  "ziemlichen",
  "ziemlicher",
  "ziemliches",
  "zirka",
  "zog",
  "zogen",
  "zu",
  "zudem",
  "zuerst",
  "zufolge",
  "zugleich",
  "zuletzt",
  "zum",
  "zumal",
  "zumeist",
  "zumindest",
  "zunächst",
  "zunaechst",
  "zur",
  "zurück",
  "zurueck",
  "zusammen",
  "zusehends",
  "zuviel",
  "zuviele",
  "zuvieler",
  "zuweilen",
  "zwanzig",
  "zwar",
  "zwei",
  "zweifelsfrei",
  "zweifelsfreie",
  "zweifelsfreiem",
  "zweifelsfreien",
  "zweifelsfreier",
  "zweifelsfreies",
  "zwischen",
  "zwölf",

  # weitere stopwords 
  "et", "et al", "al",
  "exercise", 
  "projektpraktikum",
  "ii", "qe ii", "qe"
  # "strongly", "correlated", 
  # "englischen",
  # "mögliche",
  # "eingeführt",
  # "milestones",
  # "wesentlichen",
  # "zwischen",
  # "based",
  # "diskutiert",
  # "hergestellten", "einzelne",
  # "theoretischen einweisung",
  # "ökonomischen einsatzfeldern",
]

sw = list(stopwords.get_stopwords("en"))
sw.extend(list(stopwords.get_stopwords("de")))
sw.extend(irrelevant_terms)

### Lemmatisierung
Durch Lemmatisierung werden die Wörter in einheitliche Begriffe umgewandelt, sodass diese robuster werden. 

## Nur zum Nachlesen: Konfiguration, Training und Evaluation des Topic Models
Im Folgenden werden die einzelnen Schritte erläutert, wie man automatisiert ein, gemäss dem vordefinierten Goldstandard, möglichst performantes Modell erstellen kann. 
Dies dient lediglich als Nachschlagewerk, die Code-Zellen daher bitte __NICHT__ ausführen, während man das Topic Model automatisiert erstellen lassen will.
Die Anwendung für das aktuelle Topic Modeling wird im nächsten Block durchgeführt.


### Konfiguration des Modells

#### CountVectorizer
Der `CountVectorizer` ist ein wichtiges Werkzeug bei der Vorbereitung von Textdaten für ein Topic Model wie BERTopic. Ein Topic Model analysiert große Mengen an Text, um wiederkehrende Themen (Topics) zu erkennen. BERTopic kombiniert dabei Techniken aus der Natural Language Processing (NLP) mit Clustering-Methoden, um diese Themen zu extrahieren. Der `CountVectorizer` hilft dabei, den Text in eine numerische Darstellung umzuwandeln, die für das Modell nutzbar ist.

- `stop_words=sw`:  
  Stopwörter (z. B. "und", "der", "ein"), die keine inhaltliche Bedeutung tragen, werden entfernt. Dies stellt sicher, dass das Modell nur auf relevante Begriffe fokussiert ist und keine irrelevanten Wörter in die Themenbildung einfließen.

- `token_pattern=r'\b\w+\b'`:  
  Der reguläre Ausdruck sorgt dafür, dass nur ganze Wörter als Token betrachtet werden. Sonderzeichen oder isolierte Zahlen werden ausgeschlossen, da sie selten zur inhaltlichen Bedeutung beitragen.

- `ngram_range=(1, 3)`:  
  Es werden nicht nur einzelne Wörter (1-Gramme), sondern auch Wortkombinationen aus zwei oder drei aufeinanderfolgenden Wörtern (n-Gramme) berücksichtigt. Diese Phrasen wie "künstliche Intelligenz" oder "Datenanalyse" helfen BERTopic, kontextbezogene und aussagekräftige Themen zu identifizieren.

In [5]:
vectorizer = CountVectorizer(
    stop_words=sw,  # Entfernt Stopwörter basierend auf der angegebenen Liste (sw)
    token_pattern=r'\b\w+\b',  # Extrahiert nur ganze Wörter, d. h. keine Sonderzeichen oder Zahlen
    ngram_range=(1, 3)  # Erstellt 1-Gramme (einzelne Wörter) bis 3-Gramme (Wortgruppen aus bis zu 3 aufeinanderfolgenden Wörtern)
)

#### EmbeddingSettings

Die `EmbeddingSettings` definieren die Konfiguration für die Generierung von Text-Embeddings, die zur numerischen Darstellung von Textdaten verwendet werden. Diese Embeddings fassen semantische Ähnlichkeiten und Bedeutungen von Texten in einem Vektorraum zusammen und dienen als Grundlage für weitere Analysen, z. B. Clustering oder Themenmodellierung.

### Parameter der EmbeddingSettings

- `embedding_model`:  
  Gibt das Modell an, das zur Generierung der Embeddings verwendet wird. Hier wird die Klasse `SentenceTransformer` genutzt, die leistungsstarke vortrainierte Transformer-Modelle für die Textverarbeitung unterstützt.

- `model_name_or_path`:  
  Gibt den Pfad oder Namen des vortrainierten Modells an. In diesem Fall wird das Modell `sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2` verwendet, das speziell für mehrsprachige Anwendungen optimiert ist. Dieses Modell erzeugt kompakte und semantisch aussagekräftige Embeddings.


In [6]:
EmbeddingSettings = {
    "embedding_model": SentenceTransformer,
    "model_name_or_path": "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
}

#### UMAPSettings

UMAP (Uniform Manifold Approximation and Projection) ist eine Technik zur Dimensionsreduktion, die häufig in der Verarbeitung von hochdimensionalen Daten, wie Text- oder Bilddaten, eingesetzt wird. Sie projiziert mehrdimensionale Daten in einen Raum mit geringerer Dimension, um Muster und Strukturen leichter zu erkennen. Die `UMAPSettings` definieren die Parameter, die das Verhalten und die Genauigkeit dieser Projektion steuern.


- `n_neighbors=15`:  
  Gibt die Anzahl der nächsten Nachbarn an, die für jeden Punkt berücksichtigt werden. Ein höherer Wert fokussiert auf größere Strukturen in den Daten, während ein niedrigerer Wert stärker lokale Muster betont.

- `n_components=5`:  
  Legt die Dimension des reduzierten Raumes fest. In diesem Fall werden die Daten in 5 Dimensionen projiziert, was hilft, wesentliche Eigenschaften der Daten zu erhalten.

- `min_dist=0.1`:  
  Bestimmt, wie nah Punkte im projizierten Raum beieinander liegen können. Ein niedriger Wert führt zu eng gepackten Clustern, während ein höherer Wert eine gleichmäßigere Verteilung ermöglicht.

- `metric="cosine"`:  
  Gibt die Distanzmetrik an, die verwendet wird, um die Ähnlichkeit zwischen Punkten zu berechnen. Der Kosinusabstand ist besonders geeignet für Textdaten oder hochdimensionale Vektoren.

- `random_state=13`:  
  Definiert einen Seed-Wert für den Zufallszahlengenerator, um reproduzierbare Ergebnisse sicherzustellen.


In [7]:
UMAPSettings = {
    "n_neighbors": 15,
    "n_components": 5,
    "min_dist": 0.1,
    "metric": "cosine",
    "random_state": 13
}

#### HDBSCANSettings

HDBSCAN (Hierarchical Density-Based Spatial Clustering of Applications with Noise) ist ein Clustering-Algorithmus, der besonders gut mit komplexen Datensätzen umgehen kann. Er identifiziert Cluster auf der Basis von Dichte und ermöglicht es, Datenpunkte als *Rauschen* zu klassifizieren, wenn sie nicht zu einem Cluster gehören. Die `HDBSCANSettings` definieren die Parameter, die das Verhalten des Algorithmus steuern.

##### Parameter der HDBSCANSettings

- `min_samples=10`:  
  Gibt die minimale Anzahl von Datenpunkten an, die in der Nachbarschaft eines Punktes vorhanden sein müssen, damit er als Kernpunkt eines Clusters gilt. Ein höherer Wert macht den Algorithmus empfindlicher gegenüber Rauschen.

- `gen_min_span_tree=True`:  
  Erstellt einen minimalen `span tree`, der die hierarchische Struktur der Cluster visualisiert. Dies ist nützlich für die Analyse und Interpretation der Ergebnisse.

- `prediction_data=True`:  
  Ermöglicht die Generierung zusätzlicher Daten, die für die spätere Zuordnung neuer Punkte zu den Clustern verwendet werden können.

- `min_cluster_size=100`:  
  Gibt die minimale Größe eines Clusters an. Cluster mit weniger Datenpunkten werden als Rauschen betrachtet und nicht berücksichtigt.

In [8]:
HDBSCANSettings = {
    "min_samples": 3,
    "gen_min_span_tree": True,
    "prediction_data": True,
    "min_cluster_size": 10
}

#### BERTopicSettings

Die `BERTopicSettings` definieren die Konfiguration des BERTopic-Modells, das zur Themenmodellierung verwendet wird. BERTopic identifiziert wiederkehrende Themen in Textdaten und ermöglicht eine flexible Anpassung der Ergebnisse durch verschiedene Parameter.

#### Parameter der BERTopicSettings

- `top_n_words=10`:  
  Gibt an, wie viele Schlüsselwörter pro Thema angezeigt werden. Ein höherer Wert liefert detailliertere Informationen zu den Themen.

- `language="multilingual"`:  
  Setzt die Sprache für die Verarbeitung von Textdaten. Mit "multilingual" wird sichergestellt, dass Texte in mehreren Sprachen unterstützt werden.

- `n_gram_range=(1, 4)`:  
  Bestimmt den Bereich der n-Gramme (z. B. einzelne Wörter bis zu Vier-Wort-Kombinationen), die für die Themenanalyse berücksichtigt werden.

- `min_topic_size=100`:  
  Legt die minimale Anzahl von Dokumenten fest, die ein Thema enthalten muss, damit es berücksichtigt wird. Kleinere Mindestgrößen ermöglichen es, mehr spezialisierte Themen zu erkennen.

- `calculate_probabilities=True`:  
  Aktiviert die Berechnung von Wahrscheinlichkeiten, die die Zugehörigkeit von Dokumenten zu bestimmten Themen darstellen.

- `verbose=True`:  
  Aktiviert detaillierte Konsolenausgaben, die den Fortschritt des Modells anzeigen.

- `nr_topics=20`:  
  Setzt die Anzahl der finalen Themen auf 20. Diese Begrenzung wird durch die Reduktion ähnlicher Themen erreicht.


In [9]:
BERTopicSettings = {
    "top_n_words": 10,  # Mehr Top-Wörter, um relevantere Themen zu erfassen
    "language": "multilingual",
    "n_gram_range": (1, 3),  # Erweiterung des n-Gram-Bereichs
    "min_topic_size": 10,  # Kleinere Mindestgröße der Themen
    "calculate_probabilities": True,
    "verbose": True,
    "nr_topics": None,  # Festlegung der Anzahl der Themen
}

#### BERTopic: Initialisieren und trainieren

Im folgenden wird das Modell initialisiert.

##### Parameter BERTopic

- `ctfidf_model`:  
  Ein ClassTfidfTransformer wird mit dem Parameter `reduce_frequent_words=True` initialisiert. Dies reduziert den Einfluss hochfrequenter Wörter, die zusätzlich zu Stopwörtern das Modell verzerren könnten.

- `topic_model`:  
  Initialisierung des BERTopic-Modells mit verschiedenen benutzerdefinierten Einstellungen:
  - `ctfidf_model=ctfidf_model`:  
    Verwendet das zuvor definierte ClassTfidfTransformer-Modell, um hochfrequente Wörter zu behandeln.
  - `vectorizer_model=vectorizer`:  
    Übergibt den zuvor definierten CountVectorizer, der für die Tokenisierung und n-Gramm-Erstellung genutzt wird.
  - `embedding_model=EmbeddingSettings["embedding_model"](EmbeddingSettings["model_name_or_path"])`:  
    Erstellt ein Text-Embedding-Modell basierend auf den in EmbeddingSettings angegebenen Parametern.
  - `umap_model=UMAP(**UMAPSettings)`:  
    Nutzt UMAP zur Dimensionsreduktion mit den vorher definierten Einstellungen in UMAPSettings.
  - `hdbscan_model=HDBSCAN(**HDBSCANSettings)`:  
    Führt das Clustering mit HDBSCAN durch, basierend auf den Einstellungen in HDBSCANSettings.
  - `**BERTopicSettings`:  
    Übernimmt zusätzliche Parameter aus BERTopicSettings, wie die Anzahl der Themen oder die Sprache.

- Modelltraining und Transformation:  
  Der Datensatz wird durch das `fit_transform`-Verfahren verarbeitet:
  - `topics`: Enthält die identifizierten Themen für jedes Dokument.
  - `probs`: Liefert die Wahrscheinlichkeiten, mit denen ein Dokument zu einem Thema gehört.

- Themenanalyse:  
  `topic_model.get_topic_info()` gibt eine Übersicht der erkannten Themen, deren Häufigkeit und Repräsentation zurück. Hier werden die Top 50 Themen extrahiert und nach ihrer ID aufgelistet.


#### Definition Objective
Hier wird das Package "optuna" verwendet, durch welches die zu testenden Parameter festgelegt werden können und anhand einer definierten Metrik optimiert werden können.

## Anwendung: Konfiguration, Training und Evaluation des Topic Models
Hier muss alles innerhalb einer einzigen Code-Zelle erfolgen, da bei allen Konfigurationen variable Parameter vorkommen und wir diese durch das optuna-Package optimieren wollen.

In [10]:
# Im Objective werden die verschiedenen Parameter-Settings gesetzt, über welche man optimieren möchte
def objective(trial):

  try:
    # Embedding Settings
    embedding_model_name = trial.suggest_categorical("embedding_model", ["paraphrase-multilingual-MiniLM-L12-v2", "paraphrase-mpnet-base-v2"])
    # UMAP Settings
    n_neighbors = trial.suggest_int("n_neighbors", 4, 14, 2)
    min_dist = trial.suggest_float("min_dist", 0.0, 0.2, step = 0.1)
    n_components = trial.suggest_int("n_components", 3, 8, 1)
    # HDBSCAN Settings
    min_cluster_size = trial.suggest_int("min_cluster_size", 3, 8, 1)
    #min_samples = trial.suggest_int("min_samples", 3, 10, 1)
    # BERTopic Settings
    #nr_topics = trial.suggest_categorical("nr_topics", [5, 6, 7, 8, 9, 10, 15])
    diversity = trial.suggest_float("diversity", 0.0, 0.2, step = 0.1)


    #--------------------------------------------------------------------------------------------------------------------------------------------
    # Konfiguration
    #--------------------------------------------------------------------------------------------------------------------------------------------

    # CountVectorizer
    vectorizer = CountVectorizer(
      stop_words=sw,  # Entfernt Stopwörter basierend auf der angegebenen Liste (sw)
      token_pattern=r'\b\w+\b',  # Extrahiert nur ganze Wörter, d. h. keine Sonderzeichen oder Zahlen
      ngram_range=(1, 3)  # Erstellt 1-Gramme (einzelne Wörter) bis 3-Gramme (Wortgruppen aus bis zu 3 aufeinanderfolgenden Wörtern)
    )

    # Embedding Settings  
    embedding_model = SentenceTransformer(embedding_model_name)
    
    # UMAP Settings
    umap_model = UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=n_components, metric="cosine", random_state=13)

    # HDBSCAN Settings
    hdbscan_model = HDBSCAN(min_cluster_size=min_cluster_size, gen_min_span_tree=True, prediction_data=True)

    # Representation Settings
    representation_model = MaximalMarginalRelevance(diversity=diversity)


    #--------------------------------------------------------------------------------------------------------------------------------------------
    # Training
    #--------------------------------------------------------------------------------------------------------------------------------------------

    # BERTopic initialisieren
    topic_model = BERTopic(
      embedding_model=embedding_model,
      #min_topic_size=10,
      #nr_topics=nr_topics, 
      language="multilingual",
      umap_model=umap_model,
      vectorizer_model=vectorizer,
      hdbscan_model=hdbscan_model,
      top_n_words = 30,
      representation_model=representation_model
    )

    # BERTopic trainieren
    topic_model_quanten = topic_model.fit(docs)


    #--------------------------------------------------------------------------------------------------------------------------------------------
    # Evaluation
    #--------------------------------------------------------------------------------------------------------------------------------------------

    # BERTopic auf Test-Daten anwenden
    topics, probs = topic_model_quanten.transform(test_set)
    print(topic_model_quanten.get_topic_freq())

    # Outlier reduzieren
    topics = topic_model_quanten.reduce_outliers(test_set, topics)

    # Resultierende Topic-Nummern mit den Representations (= relevante Begriffe) zu einem Datensatz kombinieren
    dataframe_with_results_left = pd.DataFrame(topics, columns = ["Topic"])
    dataframe_with_results_right = pd.DataFrame(topic_model_quanten.get_topic_info().set_index('Topic')[['Representation']])
    dataframe_with_results = dataframe_with_results_left.join(dataframe_with_results_right, on="Topic")

    # Goldstandard (Ground Truth) mit den Ergebnissen abgleichen und Score berechnen (Score = Anteil korrekter Topic-Zuweisungen)
    row_number = 0
    metric = 0
    while row_number < len(ground_truth):
      # Den Goldstandard in eine Liste von Keywords umwandeln
      ground_truth_current_iteration = ground_truth[row_number].split(", ")
      result_current_iteration = dataframe_with_results.at[row_number, "Representation"]

      # Überprüfen, ob irgendein Begriff aus dem Resultat im Goldstandard zum Text vorkommt (1 = ja, 0 = nein)
      if any(element in result_current_iteration for element in ground_truth_current_iteration):
              metric += 1
      else: metric += 0

      row_number = row_number+1

      print(result_current_iteration)
      print(ground_truth_current_iteration)
      print("---------------------------------------------------------------------------------------------------------------")

    metric_score = metric/row_number


    #--------------------------------------------------------------------------------------------------------------------------------------------
    # Ergebnis printen und Score returnen
    #--------------------------------------------------------------------------------------------------------------------------------------------

    # Ergebnis printen
    #print("Modell evaluiert mit einem Score von ", metric_score, ". " \
    #"Verwendete Parameter: embedding model: ", embedding_model_name, ", nr_topics: ", nr_topics, ", n_neighbors: ", n_neighbors, ", min_dist: ", min_dist,
    #", n_components: ", n_components, ", min_cluster_size: ", min_cluster_size)
  
    return metric_score 
  
  except Exception as e:
      print("Trial wird aufgrund eines Errors übersprungen")
      print("Verwendete Parameter: embedding model: ", embedding_model_name, ", n_neighbors: ", n_neighbors, ", min_dist: ", min_dist,\
      ", n_components: ", n_components, ", min_cluster_size: ", min_cluster_size)
      print(e)
      raise optuna.TrialPruned()

In [11]:
study = optuna.create_study(direction = "maximize")
study.optimize(objective, n_trials = 100)

print("Best parameters:", study.best_params)

[I 2025-11-06 10:36:05,159] A new study created in memory with name: no-name-8aac78de-c363-47a5-8d0a-dd56bc6a9fd8
[I 2025-11-06 10:37:01,578] Trial 0 pruned. 


Trial wird aufgrund eines Errors übersprungen
Verwendete Parameter: embedding model:  paraphrase-multilingual-MiniLM-L12-v2 , n_neighbors:  6 , min_dist:  0.0 , n_components:  4 , min_cluster_size:  3
name 'test_set' is not defined


[W 2025-11-06 10:37:38,236] Trial 1 failed with parameters: {'embedding_model': 'paraphrase-mpnet-base-v2', 'n_neighbors': 10, 'min_dist': 0.1, 'n_components': 8, 'min_cluster_size': 3, 'diversity': 0.0} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\mhu\miniconda3\envs\bertopic-env\lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\mhu\AppData\Local\Temp\ipykernel_18168\2879465152.py", line 61, in objective
    topic_model_quanten = topic_model.fit(docs)
  File "c:\Users\mhu\miniconda3\envs\bertopic-env\lib\site-packages\bertopic\_bertopic.py", line 364, in fit
    self.fit_transform(documents=documents, embeddings=embeddings, y=y, images=images)
  File "c:\Users\mhu\miniconda3\envs\bertopic-env\lib\site-packages\bertopic\_bertopic.py", line 431, in fit_transform
    embeddings = self._extract_embeddings(
  File "c:\Users\mhu\miniconda3\envs\bertopic-env\lib\

KeyboardInterrupt: 